<a href="https://colab.research.google.com/github/parkdoyun/google_colab/blob/master/practice_pdy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  x = torch.empty(5, 3)
  y = torch.ones_like(x, device=device)
  x = x.to(device)
  z = x * y
  print(z)
  print(z.to('cpu', torch.double))


tensor([[ 7.1166e+10,  3.0928e-41,  1.7096e-43],
        [ 4.4842e-44,  8.5479e-44,  4.4842e-44],
        [ 1.6816e-43,  4.4842e-44,  5.8855e-44],
        [ 4.4842e-44,  1.6956e-43,  7.1846e+22],
        [ 9.2198e-39,  7.0374e+22, -1.3625e+37]], device='cuda:0')
tensor([[ 7.1166e+10,  3.0928e-41,  1.7096e-43],
        [ 4.4842e-44,  8.5479e-44,  4.4842e-44],
        [ 1.6816e-43,  4.4842e-44,  5.8855e-44],
        [ 4.4842e-44,  1.6956e-43,  7.1846e+22],
        [ 9.2198e-39,  7.0374e+22, -1.3625e+37]], dtype=torch.float64)


In [ ]:
# CNN 이용한 Fashion MNIST 데이터 학습시키기 연습
# CNN 이용해서 의류 관련 데이터 학습 -> 최종 의류 데이터 분류
# 1. Fashion MNIST data load하기
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
# 2. 간단 CNN 모델 구현
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))  

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))

  return model

In [ ]:
# 3. model 학습
import os

model = create_model()
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17, # 17번의 epoch 동안 학습 진행
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

model.save_weights('./fashion_mnist.h5', overwrite=True)

Epoch 1/17
60/60 [==============================] - 46s 231ms/step - loss: 1.1929 - sparse_categorical_accuracy: 0.6806
Epoch 2/17
60/60 [==============================] - 14s 230ms/step - loss: 0.5301 - sparse_categorical_accuracy: 0.8205
Epoch 3/17
60/60 [==============================] - 14s 230ms/step - loss: 0.4441 - sparse_categorical_accuracy: 0.8465
Epoch 4/17
60/60 [==============================] - 14s 230ms/step - loss: 0.3868 - sparse_categorical_accuracy: 0.8640
Epoch 5/17
60/60 [==============================] - 14s 230ms/step - loss: 0.3471 - sparse_categorical_accuracy: 0.8779
Epoch 6/17
60/60 [==============================] - 14s 229ms/step - loss: 0.3168 - sparse_categorical_accuracy: 0.8870
Epoch 7/17
60/60 [==============================] - 14s 230ms/step - loss: 0.2894 - sparse_categorical_accuracy: 0.8974
Epoch 8/17
60/60 [==============================] - 14s 229ms/step - loss: 0.2704 - sparse_categorical_accuracy: 0.9022
Epoch 9/17
60/60 [======================

In [ ]:
# 4. test
from matplotlib import pyplot
%matplotlib inline

def plot_predictions(images, predictions):
  n = images.shape[0]
  nc = int(np.ceil(n/4))
  f, axes = pyplot.subplots(nc, 4)
  for i in range(nc * 4):
    y = i // 4
    x = i % 4
    axes [x, y].axis('off')

    label = LABEL_NAMES[np.argmax(predictions[i])]
    confidence = np.max(predictions[i])

    if i > n:
      continue
    axes[x, y].imshow(images[i])
    axes[x,y].text(0.5, 0.5, label + '\n%.3f' % confidence, fontsize = 14)

  pyplot.gcf().set_size_inches(8, 8)

LABEL_NAMES = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']

cpu_model = create_model()
cpu_model.load_weights('./fashion_mnist.h5')

plot_predictions(np.squeeze(x_test[:16]),
                 cpu_model.predict(x_test[:16]))